In [3]:
search_for = 125
start_from = 0
threads = 10
thread = 2
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.00452733039855957
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 35
all cases: 320327931
type: [1, 1, 1, 1, 125] 125
cases of this type: 244140625
100000 19.13405914389604
200000 26.625638972105534
300000 24.71607900405124
400000 25.683989092441365
500000 24.800872562705827
600000 25.02335894834914
700000 24.84487298271872
800000 24.675458449136354
900000 25.24599900999268
1000000 12.047312657951258
1100000 22.560703015187638
1200000 23.758687521537514
1300000 23.7711673436944
1400000 24.43650792406657
1500000 25.14847281621056
1600000 23.83596196026704
1700000 25.036204819356016
1800000 23.378287428054488
1900000 25.130017080388907
2000000 8.224612467907342
2100000 26.458707848694743
2200000 28.315365869267808
2300000 28.684993501092006
2400000 27.17144703086362
2500000 26.818563354459513
2600000 26.303202848948473
2700000 26.058103310903466
2800000 26.11930548910264
2900000 25.13421441700931
3000000 11.79

29400000 23.78465915507135
29500000 23.963117514638746
29600000 9.64069387516412
29700000 23.155351830822458
29800000 22.298985074834494
29900000 22.53864875804694
30000000 21.539538422915356
30100000 20.246073094308766
30200000 20.437856639416342
30300000 9.76871423397877
30400000 19.68892179478288
30500000 21.013137994502365
30600000 21.00988923239353
30700000 21.513301388035842
30800000 22.327989105432536
30900000 22.845669592790077
31000000 17.062961013120972
31100000 16.176013991932102
31200000 22.610460564196302
31300000 22.85280659889402
31400000 22.841492267272923
31500000 18.311678679683183
31600000 15.30871891965059
31700000 23.674911371325976
31800000 22.817312027256847
31900000 22.72330446155252
32000000 21.764140668796927
32100000 20.377048834092317
32200000 19.620823077516906
32300000 10.09740881248351
32400000 20.484840708048715
32500000 20.85544603689128
32600000 22.272513949596277
32700000 21.22968668984639
32800000 24.48867451437287
32900000 23.065255312612024
3300000

59100000 10.315795257033805
59200000 17.00412798241443
59300000 19.509875634777195
59400000 19.610738143740388
59500000 19.09318719819561
59600000 8.609573079344173
59700000 18.86447947773962
59800000 18.643355462046834
59900000 18.830202354514583
60000000 18.280977864203713
60100000 10.173844596907031
60200000 19.35031557141292
60300000 20.268711075984246
60400000 20.694639465819204
60500000 21.253257579304705
60600000 22.450180792627982
60700000 20.274191968603127
60800000 20.044022535161556
60900000 19.226317191010164
61000000 21.141025150069684
61100000 8.022824082372031
61200000 17.480676829268862
61300000 18.23264033079048
61400000 17.777979472912065
61500000 16.132472630238773
61600000 9.601197202008047
61700000 17.321343146484534
61800000 18.81197477779917
61900000 17.926936658675096
62000000 16.043011683895834
62100000 13.536260489854747
62200000 19.161769553440536
62300000 20.569762055265905
62400000 19.376688966766764
62500000 21.123858844111016
62600000 21.05640400942502
62

88900000 6.243342867601086
89000000 15.216060333195317
89100000 15.112844037277055
89200000 8.88176758861555
89300000 16.093361303840496
89400000 15.53742362822442
89500000 16.23872566816782
89600000 17.49034910034421
89700000 17.179027240865004
89800000 17.749678791447398
89900000 18.266959992630735
90000000 17.636557831161117
90100000 17.470382984386788
90200000 16.329704046575163
90300000 16.940936896989314
90400000 15.498091794336508
90500000 15.747334844255159
90600000 8.049570642686026
90700000 15.182764299786266
90800000 14.708659795666314
90900000 8.292076715811843
91000000 15.616719078251817
91100000 9.062527753441593
91200000 14.712886462682789
91300000 16.696337421509803
91400000 16.364357377052418
91500000 15.835081766414238
91600000 17.085271732857887
91700000 17.488211852181937
91800000 16.857398632871885
91900000 17.648269827802334
92000000 17.748650320952354
92100000 16.293704160520203
92200000 15.791400005765773
92300000 16.800180718623885
92400000 16.09823353277528
92

117900000 1.5249666726558366
118000000 1.5612760948275786
118100000 1.5673755969884087
118200000 1.5980199055242896
118300000 1.5961748467094654
118400000 1.682005392245692
118500000 1.5557503383232911
118600000 1.5471347009808984
118700000 1.5362331465876757
118800000 1.52931201162584
118900000 1.5021045133140325
119000000 1.5042429378135054
119100000 1.534842141608238
119200000 1.6634849185256562
119300000 1.5665080206577744
119400000 1.5367227226046243
119500000 1.6855637677900384
119600000 1.5938738178400769
119700000 1.559941437643828
119800000 1.739465404776368
119900000 1.613859254190104
120000000 1.6830944866554916
120100000 1.5586250770262413
120200000 1.586967963622138
120300000 1.6182117911779246
120400000 1.485366236396261
120500000 1.6545630452593558
120600000 1.6701296921663304
120700000 1.4759272690590395
120800000 1.5590841737195782
120900000 1.5144506828746915
121000000 1.5777072638974003
121100000 1.7183921094871641
121200000 1.545054228384014
121300000 1.557349486697

...

204300000 7.890849852932621
204400000 7.729084284722336
204500000 8.799054476780778
204600000 8.209270268815924
204700000 8.486631827907447
204800000 3.8137170165677805
204900000 8.872136998783372
205000000 8.54182100727149
205100000 8.641061936080165
205200000 8.427004208973688
205300000 8.295128488600287
205400000 5.132981591864603
205500000 7.6121953106586995
205600000 8.204505758658895
205700000 7.858785153620071
205800000 7.924562182057215
205900000 7.375751849557072
206000000 7.37455724704021
206100000 3.012348291287033
206200000 7.619349935151461
206300000 7.60814030739479
206400000 7.7329760028368355
206500000 7.9799165642685095
206600000 8.04838559335134
206700000 7.900580251675291
206800000 4.426687584342947
206900000 8.35311771311736
207000000 8.333500576170394
207100000 8.511297275383301
207200000 8.312744072617187
207300000 7.519457118415672
207400000 5.247874672304194
207500000 8.679426405870487
207600000 8.005301342984085
207700000 8.020590120195012
207800000 7.824542316

233600000 6.378650107546249
233700000 6.275769400862478
233800000 6.109135735079085
233900000 6.3222437043904005
234000000 6.610702397582019
234100000 6.413254414991095
234200000 6.504944296202184
234300000 4.13928031650238
234400000 5.501733827407151
234500000 3.229395630458444
234600000 6.582368192116259
234700000 6.334952388006391
234800000 6.464135904030535
234900000 6.544581992389455
235000000 6.183150907371098
235100000 6.226506988701406
235200000 6.39173335110541
235300000 6.2896491164662525
235400000 2.2668224021542973
235500000 6.544724970348333
235600000 6.485895474945537
235700000 6.274981342702087
235800000 6.383235090127261
235900000 6.669882102786414
236000000 6.5521138397121765
236100000 6.480688571191406
236200000 6.4900616310253625
236300000 2.782980452443012
236400000 4.037166260287388
236500000 5.262833936128427
236600000 6.174625761851652
236700000 6.154019697092405
236800000 6.235660610632121
236900000 6.0394151138084045
237000000 5.8579009222521075
237100000 5.893

262200000 1.8181808769351302
262300000 1.6825213960682974
262400000 1.5832177264699956
262500000 2.0682973842665278
262600000 1.1129759213060861
262700000 0.4811313643392371
262800000 0.4854747303276142
262900000 1.0754801653475703
263000000 2.237081654438295
263100000 1.6476207341023743
263200000 1.7498809139461093
263300000 1.7955606226772247
263400000 1.610361674944947
263500000 1.77835152776187
263600000 1.752255188067972
263700000 2.0502557400723336
263800000 1.8175169456656246
263900000 1.802538054070392
264000000 1.6624238792517723
264100000 1.6388478246073213
264200000 1.599131719392613
264300000 1.6917987628635347
264400000 1.803192576803859
264500000 1.4705502496692133
264600000 0.7751457837351302
264700000 0.46350154636963214
264800000 0.6373798328681005
264900000 1.475560715276009
265000000 1.9850499018502916
265100000 1.6743731584686452
265200000 1.5319474462866804
265300000 1.5943547071182271
265400000 1.6600792584746962
265500000 1.6930368826328597
265600000 1.7101370069

290600000 0.903587408115232
290700000 0.9523268020697911
290800000 0.9889925724938584
290900000 0.9595261351307837
291000000 1.0410021484282732
291100000 1.0238298612926138
291200000 1.015680518244158
291300000 0.8712390248517573
291400000 0.94192547036529
291500000 0.995185585181927
291600000 0.8719071177847604
291700000 0.79847711666003
291800000 1.0090675000169556
291900000 0.55737183951841
292000000 0.24948480651825863
292100000 0.24445075764678914
292200000 0.5562432819458722
292300000 1.0317892588807336
292400000 0.7731737600945797
292500000 0.9089532465459744
292600000 0.9467818532598006
292700000 1.006101227253414
292800000 0.9179115471525013
292900000 0.8934907915154172
best so far: 0
type: [1, 1, 1, 25, 5] 125
cases of this type: 9765625
293000000 0.9806737960240132
293100000 0.8255073013965825
293200000 0.9043813081338453
293300000 0.7783962527407062
293400000 0.7613746867067258
293500000 0.6695292842739171
293600000 0.5965947240211129
293700000 0.6232850678671896
293800000 

317100000 0.023331794064527754
best so far: 0
type: [1, 1, 25, 5, 1] 125
cases of this type: 78125
317200000 0.022389077371155554
best so far: 0
type: [1, 1, 125, 1, 1] 125
cases of this type: 15625
best so far: 0
type: [1, 5, 1, 1, 25] 125
cases of this type: 1953125
317300000 0.029113003366343176
317400000 0.05849825451228221
317500000 0.04490313722859753
317600000 0.049324230199534365
317700000 0.04252898425991535
317800000 0.041396776884986955
317900000 0.04398475165763432
318000000 0.04638509884656071
318100000 0.031215229019449817
318200000 0.015093384455674222
318300000 0.01461051664981246
318400000 0.013003086403777864
318500000 0.025054622204226256
318600000 0.03437745490154822
318700000 0.02940203717085653
318800000 0.029265749971041415
318900000 0.02390189332946539
319000000 0.02442094904759129
319100000 0.021913368982688586
319200000 0.016979596981590985
best so far: 0
type: [1, 5, 1, 5, 5] 125
cases of this type: 390625
319300000 0.009987548633760876
319400000 0.0073980829